In [1]:
import datetime as dt
import requests
import random
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

import warnings

# Routine Update for METS Online

In [2]:
file_path = ".\export-data-malaysia.csv"
old_df = pd.read_csv(file_path)
old_df

,YEAR,FOOD,BEVERAGES AND TOBACCO,"CRUDE MATERIALS, INEDIBLE","MINERAL FUELS, LUBRICANTS, ETC.",ANIMAL AND VEGETABLE OILS AND FATS,CHEMICALS,MANUFACTURED GOODS,MACHINERY & TRANSPORT EQUIPMENT,MISCELLANEOUS MANUFACTURED ARTICLES,MISCELLANEOUS TRANSACTIONS AND COMMODITIES
0,Jan 2000,491864874,92332416,863994681,2843136683,1116946684,880219925,1927880545,14958386186,2026711654,229883181
1,Feb 2000,425641394,77214349,689603279,2881747034,1099756943,1072302224,1910675351,16986581083,1922449106,217203495
2,Mar 2000,553169207,100950924,901427068,3188319069,1254551970,1086130053,2133213423,20139482974,2368704586,262696514
3,Apr 2000,509671983,98515223,897853450,2455441551,1116557420,1087280719,2174508072,17989050119,2333513182,240322014
4,May 2000,553327793,111185482,909523666,2651869264,1008684467,1231347129,2291918346,18776956887,2480702004,253977343
...,...,...,...,...,...,...,...,...,...,...,...
277,Feb 2023,3539279631,222258336,2346324739,20323875331,5629973235,9036890095,10081396919,49850907614,10783053535,454585466
278,Mar 2023,4044164918,326561786,2715999243,21429738762,7647033259,9950967381,11464714044,57828441926,13401993529,858633228
279,Apr 2023,3251410911,229753325,1946644340,16984271124,5828048670,7583766017,10194652518,49009132271,9527059374,638171651
280,May 2023,3898340823,270752156,2454013437,21965803909,5767124163,9214928157,10424167008,52998110557,11859102111,657960595


In [3]:
# URL to the data extraction site
url = "https://metsonline.dosm.gov.my/tradev2/product-coderesult"

# Obtain csrf_token from the website
csrf_token = 'dG5PZ09oay0nPiIMATBTeTsZHFAgKSJdRQQVKwRYJBQADRckHStGYA=='

# Create headers for all posts
headers = {
"Content-Type": "application/x-www-form-urlencoded; charset=UTF-8", 
"Cookie": "_ga_F8QG3XBL63=GS1.1.1679998862.1.1.1679998930.0.0.0; _ga_7Q47WMYPD5=GS1.1.1679998865.1.1.1679998930.0.0.0; _ga_HK905W412V=GS1.1.1679998865.1.1.1679998930.0.0.0; mp_f55a9b6a137ab21675ef8724f7864bde_mixpanel=%7B%22distinct_id%22%3A%20%2218727cb0e099da-0247f50f77e9aa-7a545471-144000-18727cb0e0a31%22%2C%22%24device_id%22%3A%20%2218727cb0e099da-0247f50f77e9aa-7a545471-144000-18727cb0e0a31%22%2C%22%24search_engine%22%3A%20%22google%22%2C%22%24initial_referrer%22%3A%20%22https%3A%2F%2Fwww.google.com%2F%22%2C%22%24initial_referring_domain%22%3A%20%22www.google.com%22%2C%22__timers%22%3A%20%7B%7D%7D; _ga_TF34V4E8EM=GS1.1.1679999831.1.1.1680004133.0.0.0; _ga_XSB7QCQ0M9=GS1.1.1681551362.3.1.1681551368.0.0.0; _ga=GA1.1.324035783.1679998837; PHPSESSID=3oi1at9dfj4ugn6nno2bsqdd60; _csrf=193a3c282709e8f8d32929d94372b30d2cc2a5402306edc5e12c04ca098a1819a%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22SPmkNX8TOwS7oAIp1jZLK0O9tcXCRC-M%22%3B%7D; BNES_PHPSESSID=fk0GuBGEQJmmIMfAn9TJ6EK+FW/oxjgTr2AAO+vNTw1JHh1+YzlOkigmNmbMOtjxuyDVXvBqWx9L6ZHOVdyK0Unb9r8ZsuhgKRw0f3zlmA8=; BNES__csrf=T1JRVt/fspwmarDUSua9ANdq7/OTqhlhu8lprfzaWWaQtpaDzljVNyJUL1Y65guWawDI8LDmCHrD6nlIXud7o3ytcCBG5TDvi4Kd94SBmSCx1ED1HWM0YV+ztPXAQ5AhjCkcJExisSE5tRQSObfAv0WAFW/4frqxEg4pTyBtkcJZTi4Amki6jSChfqJAyShFuYOrg/ZKI0IsB8ShUR5UFcxqa3a28w/OAICYVhchalmoAAhmCMY22tozvta3gpBJGC+pONVCkNPozWC+7XVzDC5ZciTE34Uq"
}

In [4]:
# To obtain the current datetime
current_year = dt.date.today().year
targeted_year = []
targeted_year.append(current_year)
targeted_year

[2023]

In [ ]:
# To start extract data from url via request
data_raw = []

max_retries = 2

for year in targeted_year:
    payload_month = {
    '_csrf': csrf_token,
    'Tradev2[typeofsearch]': 'classification',
    'Tradev2[typedigit]': 7,
    'Tradev2[rangecode1]': 0,
    'Tradev2[rangecode2]': 9,
    # 'Tradev2[code_idcode]': ,
    # 'Tradev2[code_idcodedigit9]': ,
    # 'Tradev2[tradeflow]': ,
    'Tradev2[tradeflow][]': 'exports',
    # 'Tradev2[timeframe]': ,
    'Tradev2[timeframe]': 'month',
    # 'Tradev2[rangeyear]': ,
    # 'Tradev2[rangeyear2]': ,
    # 'Tradev2[rangeyearone]': ,
    # 'Tradev2[rangemonthone]': ,
    'Tradev2[mothdata]': year,
    'Tradev2[mothdata2]': year,
    # 'Tradev2[classification_serch]': ,
    # 'Tradev2[country2]': ,
    'Tradev2[geogroup]': 1,
    'Tradev2[geogroup]': 29,
    'Tradev2[codeshowby]': 'code'
    }
    
    for retry in range(max_retries):
        retry_delay = random.randint(10,15)  # seconds
        try:
            # Process the response here
            monthly_raw = requests.post(url, data=payload_month, headers=headers)
            data_raw.append(monthly_raw) # Store the extracted raw data into a list
            print(f"Routine Data Extraction for year {year} is Successful.")
            time.sleep(5)
            break # Break the retry loop if successful
            
        except (requests.ConnectionError) as e:
            print(f"Error: {e}. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
            
        except Exception as e:
            print(f"An error occurred: {e}")
            break  # Stop retrying if a different error occurs
     
print(data_raw)